# LLM Optimization Modelling Experiment

In [8]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [237]:
problem = '''You are task with maximizing the water flow in a network of pipes over 10 time steps. The water enters the system at point A and exits the system at point G. Hence, you need to maximize the amount of water that flows to point G. The water enters point A according to the function f(t) = max(-4/30t^3+t^2-0.234t+3, 0), where t is the time step. There are serveral points that are connected through pipes. Each pipe has a fixed capacity. Each point has to possibility to release water out of the system in case the outgoing pipes are full. There are 12 available pipes in total but only 10 of them can be activated. Deactivated pipes can not be used to transport water. The activation status of a pipe is set before any water arrives at point A and does not change. Finally, water can not be stored at the connection points which means that any water coming in at time t needs to be flowing out at time t + 1.

These are the pipe capacities:
AB: 3
AC: 6
AF: 1
BC: 3
BD: 10
BE: 4
CE: 4
CF: 4
DE: 5
EG: 5
FB: 12
FG: 7'''

## 2. Ask for parameters

In [238]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [239]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Variables:

**1. Pipe Activation Status (Binary)**

*  `x_AB`, `x_AC`, `x_AF`, `x_BC`, `x_BD`, `x_BE`, `x_CE`, `x_CF`, `x_DE`, `x_EG`, `x_FB`, `x_FG` 
    *  `1` if the pipe is activated, `0` otherwise.

**2. Water Flow in Pipes (Continuous, Non-negative)**

*  `f_AB(t)`, `f_AC(t)`, `f_AF(t)`, `f_BC(t)`, `f_BD(t)`, `f_BE(t)`, `f_CE(t)`, `f_CF(t)`, `f_DE(t)`, `f_EG(t)`, `f_FB(t)`, `f_FG(t)`
    * Represents the water flow through the respective pipe at time step `t` where `t = 1, 2,..., 10`.

**3. Water Released at Points (Continuous, Non-negative)**

*  `r_B(t)`, `r_C(t)`, `r_D(t)`, `r_E(t)`, `r_F(t)`
    * Represents the amount of water released at the respective point at time step `t` where `t = 1, 2,..., 10`. 


# 2. Ask for objective

In [240]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [241]:
Markdown(response2.text)

Maximize:  

∑_{t=1}^{10} f_EG(t) 


# 3. Ask for constraints

In [242]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [243]:
Markdown(response3.text)

## Constraints:

**1. Pipe Activation Limit:**

*  ∑_{i ∈ {AB, AC, AF, BC, BD, BE, CE, CF, DE, EG, FB, FG}} x_i = 10 

**2. Pipe Capacity Constraints:**

*  0 ≤ f_AB(t) ≤ 3 * x_AB  for t = 1, 2,..., 10
*  0 ≤ f_AC(t) ≤ 6 * x_AC  for t = 1, 2,..., 10
*  0 ≤ f_AF(t) ≤ 1 * x_AF  for t = 1, 2,..., 10
*  0 ≤ f_BC(t) ≤ 3 * x_BC  for t = 1, 2,..., 10
*  0 ≤ f_BD(t) ≤ 10 * x_BD for t = 1, 2,..., 10
*  0 ≤ f_BE(t) ≤ 4 * x_BE  for t = 1, 2,..., 10
*  0 ≤ f_CE(t) ≤ 4 * x_CE  for t = 1, 2,..., 10
*  0 ≤ f_CF(t) ≤ 4 * x_CF  for t = 1, 2,..., 10
*  0 ≤ f_DE(t) ≤ 5 * x_DE  for t = 1, 2,..., 10
*  0 ≤ f_EG(t) ≤ 5 * x_EG  for t = 1, 2,..., 10
*  0 ≤ f_FB(t) ≤ 12 * x_FB for t = 1, 2,..., 10
*  0 ≤ f_FG(t) ≤ 7 * x_FG  for t = 1, 2,..., 10

**3. Flow Conservation at Each Point:**

*  f_AB(t) + f_AC(t) + f_AF(t) = f_BC(t-1) + f_BD(t-1) + f_BE(t-1) + r_B(t-1)  for t = 2, 3,..., 10
*  f_BC(t) + f_CE(t) + f_CF(t) = f_AC(t-1) + r_C(t-1)   for t = 2, 3,..., 10
*  f_BD(t) + f_DE(t) = f_EG(t-1) + r_D(t-1) for t = 2, 3,..., 10
*  f_BE(t) = f_DE(t-1) + r_E(t-1) for t = 2, 3,..., 10
*  f_CF(t) + f_AF(t) + f_FB(t) = f_FG(t-1) + r_F(t-1) for t = 2, 3,..., 10

**4. Initial and Final Conditions:**

* f_AB(1) + f_AC(1) + f_AF(1) = max(-4/30*1^3+1^2-0.234*1+3, 0) 
*  f_EG(10) =  f_DE(9) + f_FG(9) 

**5. Non-negativity of Released Water:**

*  r_B(t) ≥ 0 for t = 1, 2,..., 9
*  r_C(t) ≥ 0 for t = 1, 2,..., 9
*  r_D(t) ≥ 0 for t = 1, 2,..., 9
*  r_E(t) ≥ 0 for t = 1, 2,..., 9
*  r_F(t) ≥ 0 for t = 1, 2,..., 9 


## 3. Generate the pyomo code

In [244]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [245]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Create a concrete model
model = pyo.ConcreteModel()

# Define time periods
model.T = pyo.RangeSet(1, 10)

# Define pipes
pipes = {
    "AB": 3,
    "AC": 6,
    "AF": 1,
    "BC": 3,
    "BD": 10,
    "BE": 4,
    "CE": 4,
    "CF": 4,
    "DE": 5,
    "EG": 5,
    "FB": 12,
    "FG": 7,
}

# Define points
points = ["B", "C", "D", "E", "F"]

# ==============================================
# Define variables
# ==============================================

# Binary variable for pipe activation status
model.x = pyo.Var(pipes, domain=pyo.Binary)

# Continuous variable for water flow in pipes
model.f = pyo.Var(pipes, model.T, domain=pyo.NonNegativeReals)

# Continuous variable for water released at points
model.r = pyo.Var(points, model.T, domain=pyo.NonNegativeReals)

# ==============================================
# Define constraints
# ==============================================

# 1. Pipe Activation Limit:
def pipe_activation_limit_rule(model):
    return sum(model.x[i] for i in pipes) == 10

model.pipe_activation_limit = pyo.Constraint(rule=pipe_activation_limit_rule)

# 2. Pipe Capacity Constraints:
def pipe_capacity_rule(model, i, t):
    return model.f[i, t] <= pipes[i] * model.x[i]

model.pipe_capacity = pyo.Constraint(pipes, model.T, rule=pipe_capacity_rule)

# 3. Flow Conservation at Each Point:
def flow_conservation_b_rule(model, t):
    if t >= 2:
        return (
            model.f["AB", t]
            + model.f["AC", t]
            + model.f["AF", t]
            == model.f["BC", t - 1]
            + model.f["BD", t - 1]
            + model.f["BE", t - 1]
            + model.r["B", t - 1]
        )
    else:
        return pyo.Constraint.Skip

model.flow_conservation_b = pyo.Constraint(model.T, rule=flow_conservation_b_rule)

def flow_conservation_c_rule(model, t):
    if t >= 2:
        return (
            model.f["BC", t]
            + model.f["CE", t]
            + model.f["CF", t]
            == model.f["AC", t - 1]
            + model.r["C", t - 1]
        )
    else:
        return pyo.Constraint.Skip

model.flow_conservation_c = pyo.Constraint(model.T, rule=flow_conservation_c_rule)

def flow_conservation_d_rule(model, t):
    if t >= 2:
        return (
            model.f["BD", t] + model.f["DE", t] == model.f["EG", t - 1] + model.r["D", t - 1]
        )
    else:
        return pyo.Constraint.Skip

model.flow_conservation_d = pyo.Constraint(model.T, rule=flow_conservation_d_rule)

def flow_conservation_e_rule(model, t):
    if t >= 2:
        return model.f["BE", t] == model.f["DE", t - 1] + model.r["E", t - 1]
    else:
        return pyo.Constraint.Skip

model.flow_conservation_e = pyo.Constraint(model.T, rule=flow_conservation_e_rule)

def flow_conservation_f_rule(model, t):
    if t >= 2:
        return (
            model.f["CF", t]
            + model.f["AF", t]
            + model.f["FB", t]
            == model.f["FG", t - 1]
            + model.r["F", t - 1]
        )
    else:
        return pyo.Constraint.Skip

model.flow_conservation_f = pyo.Constraint(model.T, rule=flow_conservation_f_rule)

# 4. Initial and Final Conditions:
def initial_condition_rule(model):
    # This is sample data. You can change it as needed.
    return model.f["AB", 1] + model.f["AC", 1] + model.f["AF", 1] == max(-4/30*1**3+1**2-0.234*1+3, 0) 

model.initial_condition = pyo.Constraint(rule=initial_condition_rule)

def final_condition_rule(model):
    return model.f["EG", 10] == model.f["DE", 9] + model.f["FG", 9]

model.final_condition = pyo.Constraint(rule=final_condition_rule)

# ==============================================
# Define objective function
# ==============================================
def objective_rule(model):
    return sum(model.f["EG", t] for t in model.T)

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Solve the model
solver = pyo.SolverFactory("cbc")  # You can use other solvers like 'gurobi' if available
results = solver.solve(model)

# Print the results
print(results)
print("Objective Value:", pyo.value(model.objective))
```

**Sample Data Explanation:**

*  **Initial Condition:** The equation `f_AB(1) + f_AC(1) + f_AF(1) = max(-4/30*1^3+1^2-0.234*1+3, 0)` is a sample initial condition. You can replace this with the actual equation representing the water inflow at the beginning.

**Important Notes:**

* **Solver:** This code uses the open-source 'cbc' solver. You can change this to other solvers (like 'gurobi', 'cplex') by modifying the line `solver = pyo.SolverFactory('cbc')`. Make sure the solver you choose is installed and properly configured.
* **Data:** This code assumes the problem data (pipe capacities, initial conditions, etc.) is directly incorporated into the code. For larger problems, it's more efficient to load data from external files. 
* **Results:** After solving, the code will print the solver status and the objective function's optimal value. You can access the values of the decision variables (pipe activations, flow rates) using `pyo.value(model.x[i])`, `pyo.value(model.f[i, t])`, and `pyo.value(model.r[p, t])`.

This code provides a structured framework to solve the water flow optimization problem. Remember to adapt the data and potentially the constraints to fit your specific scenario.


## 4. Input problem data and try running the generated code

In [247]:
import pyomo.environ as pyo

# Create a concrete model
model = pyo.ConcreteModel()

# Define time periods
model.T = pyo.RangeSet(1, 10)

# Define pipes
pipes = {
    "AB": 3,
    "AC": 6,
    "AF": 1,
    "BC": 3,
    "BD": 10,
    "BE": 4,
    "CE": 4,
    "CF": 4,
    "DE": 5,
    "EG": 5,
    "FB": 12,
    "FG": 7,
}

# Define points
points = ["B", "C", "D", "E", "F"]

# ==============================================
# Define variables
# ==============================================

# Binary variable for pipe activation status
model.x = pyo.Var(pipes, domain=pyo.Binary)

# Continuous variable for water flow in pipes
model.f = pyo.Var(pipes, model.T, domain=pyo.NonNegativeReals)

# Continuous variable for water released at points
model.r = pyo.Var(points, model.T, domain=pyo.NonNegativeReals)

# ==============================================
# Define constraints
# ==============================================

# 1. Pipe Activation Limit:
def pipe_activation_limit_rule(model):
    return sum(model.x[i] for i in pipes) == 10

model.pipe_activation_limit = pyo.Constraint(rule=pipe_activation_limit_rule)

# 2. Pipe Capacity Constraints:
def pipe_capacity_rule(model, i, t):
    return model.f[i, t] <= pipes[i] * model.x[i]

model.pipe_capacity = pyo.Constraint(pipes, model.T, rule=pipe_capacity_rule)

# 3. Flow Conservation at Each Point:
def flow_conservation_b_rule(model, t):
    if t >= 2:
        return (
            model.f["AB", t]
            + model.f["AC", t]
            + model.f["AF", t]
            == model.f["BC", t - 1]
            + model.f["BD", t - 1]
            + model.f["BE", t - 1]
            + model.r["B", t - 1]
        )
    else:
        return pyo.Constraint.Skip

model.flow_conservation_b = pyo.Constraint(model.T, rule=flow_conservation_b_rule)

def flow_conservation_c_rule(model, t):
    if t >= 2:
        return (
            model.f["BC", t]
            + model.f["CE", t]
            + model.f["CF", t]
            == model.f["AC", t - 1]
            + model.r["C", t - 1]
        )
    else:
        return pyo.Constraint.Skip

model.flow_conservation_c = pyo.Constraint(model.T, rule=flow_conservation_c_rule)

def flow_conservation_d_rule(model, t):
    if t >= 2:
        return (
            model.f["BD", t] + model.f["DE", t] == model.f["EG", t - 1] + model.r["D", t - 1]
        )
    else:
        return pyo.Constraint.Skip

model.flow_conservation_d = pyo.Constraint(model.T, rule=flow_conservation_d_rule)

def flow_conservation_e_rule(model, t):
    if t >= 2:
        return model.f["BE", t] == model.f["DE", t - 1] + model.r["E", t - 1]
    else:
        return pyo.Constraint.Skip

model.flow_conservation_e = pyo.Constraint(model.T, rule=flow_conservation_e_rule)

def flow_conservation_f_rule(model, t):
    if t >= 2:
        return (
            model.f["CF", t]
            + model.f["AF", t]
            + model.f["FB", t]
            == model.f["FG", t - 1]
            + model.r["F", t - 1]
        )
    else:
        return pyo.Constraint.Skip

model.flow_conservation_f = pyo.Constraint(model.T, rule=flow_conservation_f_rule)

# 4. Initial and Final Conditions:
def initial_condition_rule(model):
    # This is sample data. You can change it as needed.
    return model.f["AB", 1] + model.f["AC", 1] + model.f["AF", 1] == max(-4/30*1**3+1**2-0.234*1+3, 0) 

model.initial_condition = pyo.Constraint(rule=initial_condition_rule)

def final_condition_rule(model):
    return model.f["EG", 10] == model.f["DE", 9] + model.f["FG", 9]

model.final_condition = pyo.Constraint(rule=final_condition_rule)

# ==============================================
# Define objective function
# ==============================================
def objective_rule(model):
    return sum(model.f["EG", t] for t in model.T)

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Solve the model
solver = pyo.SolverFactory("glpk")  # You can use other solvers like 'gurobi' if available
results = solver.solve(model)

# Print the results
print(results)
print("Objective Value:", pyo.value(model.objective))


Problem: 
- Name: unknown
  Lower bound: 50.0
  Upper bound: 50.0
  Number of objectives: 1
  Number of constraints: 168
  Number of variables: 177
  Number of nonzeros: 474
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 5
      Number of created subproblems: 5
  Error rc: 0
  Time: 0.05135345458984375
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Objective Value: 50.0


## 5. Correct the code to verify model viability (optional)

In [248]:
import pyomo.environ as pyo

# Create a concrete model
model = pyo.ConcreteModel()

# Define time periods
model.T = pyo.RangeSet(1, 10)

# Define pipes
pipes = {
    "AB": 3,
    "AC": 6,
    "AF": 1,
    "BC": 3,
    "BD": 10,
    "BE": 4,
    "CE": 4,
    "CF": 4,
    "DE": 5,
    "EG": 5,
    "FB": 12,
    "FG": 7,
}

# Define points
points = ["B", "C", "D", "E", "F"]

# ==============================================
# Define variables
# ==============================================

# Binary variable for pipe activation status
model.x = pyo.Var(pipes, domain=pyo.Binary)

# Continuous variable for water flow in pipes
model.f = pyo.Var(pipes, model.T, domain=pyo.NonNegativeReals)

# Continuous variable for water released at points
model.r = pyo.Var(points, model.T, domain=pyo.NonNegativeReals)

# ==============================================
# Define constraints
# ==============================================

# 1. Pipe Activation Limit:
def pipe_activation_limit_rule(model):
    return sum(model.x[i] for i in pipes) == 10

model.pipe_activation_limit = pyo.Constraint(rule=pipe_activation_limit_rule)

# 2. Pipe Capacity Constraints:
def pipe_capacity_rule(model, i, t):
    return model.f[i, t] <= pipes[i] * model.x[i]

model.pipe_capacity = pyo.Constraint(pipes, model.T, rule=pipe_capacity_rule)

# 3. Flow Conservation at Each Point:
def flow_conservation_b_rule(model, t):
    if t >= 2:
        return (
            model.f["AB", t]
            + model.f["AC", t]
            + model.f["AF", t]
            == model.f["BC", t - 1]
            + model.f["BD", t - 1]
            + model.f["BE", t - 1]
            + model.r["B", t - 1]
        )
    else:
        return pyo.Constraint.Skip

model.flow_conservation_b = pyo.Constraint(model.T, rule=flow_conservation_b_rule)

def flow_conservation_c_rule(model, t):
    if t >= 2:
        return (
            model.f["BC", t]
            + model.f["CE", t]
            + model.f["CF", t]
            == model.f["AC", t - 1]
            + model.r["C", t - 1]
        )
    else:
        return pyo.Constraint.Skip

model.flow_conservation_c = pyo.Constraint(model.T, rule=flow_conservation_c_rule)

def flow_conservation_d_rule(model, t):
    if t >= 2:
        return (
            model.f["BD", t] + model.f["DE", t] == model.f["EG", t - 1] + model.r["D", t - 1]
        )
    else:
        return pyo.Constraint.Skip

model.flow_conservation_d = pyo.Constraint(model.T, rule=flow_conservation_d_rule)

def flow_conservation_e_rule(model, t):
    if t >= 2:
        return model.f["BE", t] == model.f["DE", t - 1] + model.r["E", t - 1]
    else:
        return pyo.Constraint.Skip

model.flow_conservation_e = pyo.Constraint(model.T, rule=flow_conservation_e_rule)

def flow_conservation_f_rule(model, t):
    if t >= 2:
        return (
            model.f["CF", t]
            + model.f["AF", t]
            + model.f["FB", t]
            == model.f["FG", t - 1]
            + model.r["F", t - 1]
        )
    else:
        return pyo.Constraint.Skip

model.flow_conservation_f = pyo.Constraint(model.T, rule=flow_conservation_f_rule)

# 4. Initial and Final Conditions:
def initial_condition_rule(model):
    # This is sample data. You can change it as needed.
    return model.f["AB", 1] + model.f["AC", 1] + model.f["AF", 1] == max(-4/30*1**3+1**2-0.234*1+3, 0) 

model.initial_condition = pyo.Constraint(rule=initial_condition_rule)

def final_condition_rule(model):
    return model.f["EG", 10] == model.f["DE", 9] + model.f["FG", 9]

model.final_condition = pyo.Constraint(rule=final_condition_rule)

# ==============================================
# Define objective function
# ==============================================
def objective_rule(model):
    return sum(model.f["EG", t] for t in model.T) + sum(model.f["FG", t] for t in model.T)

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Solve the model
solver = pyo.SolverFactory("glpk")  # You can use other solvers like 'gurobi' if available
results = solver.solve(model)

# Print the results
print(results)
print("Objective Value:", pyo.value(model.objective))


Problem: 
- Name: unknown
  Lower bound: 118.0
  Upper bound: 118.0
  Number of objectives: 1
  Number of constraints: 168
  Number of variables: 177
  Number of nonzeros: 474
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 7
      Number of created subproblems: 7
  Error rc: 0
  Time: 0.04165291786193848
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Objective Value: 118.0


## 6. Print the responses

In [249]:
print(response.text)

## Variables:

**1. Pipe Activation Status (Binary)**

*  `x_AB`, `x_AC`, `x_AF`, `x_BC`, `x_BD`, `x_BE`, `x_CE`, `x_CF`, `x_DE`, `x_EG`, `x_FB`, `x_FG` 
    *  `1` if the pipe is activated, `0` otherwise.

**2. Water Flow in Pipes (Continuous, Non-negative)**

*  `f_AB(t)`, `f_AC(t)`, `f_AF(t)`, `f_BC(t)`, `f_BD(t)`, `f_BE(t)`, `f_CE(t)`, `f_CF(t)`, `f_DE(t)`, `f_EG(t)`, `f_FB(t)`, `f_FG(t)`
    * Represents the water flow through the respective pipe at time step `t` where `t = 1, 2,..., 10`.

**3. Water Released at Points (Continuous, Non-negative)**

*  `r_B(t)`, `r_C(t)`, `r_D(t)`, `r_E(t)`, `r_F(t)`
    * Represents the amount of water released at the respective point at time step `t` where `t = 1, 2,..., 10`. 



In [250]:
print(response2.text)

Maximize:  

∑_{t=1}^{10} f_EG(t) 



In [251]:
print(response3.text)

## Constraints:

**1. Pipe Activation Limit:**

*  ∑_{i ∈ {AB, AC, AF, BC, BD, BE, CE, CF, DE, EG, FB, FG}} x_i = 10 

**2. Pipe Capacity Constraints:**

*  0 ≤ f_AB(t) ≤ 3 * x_AB  for t = 1, 2,..., 10
*  0 ≤ f_AC(t) ≤ 6 * x_AC  for t = 1, 2,..., 10
*  0 ≤ f_AF(t) ≤ 1 * x_AF  for t = 1, 2,..., 10
*  0 ≤ f_BC(t) ≤ 3 * x_BC  for t = 1, 2,..., 10
*  0 ≤ f_BD(t) ≤ 10 * x_BD for t = 1, 2,..., 10
*  0 ≤ f_BE(t) ≤ 4 * x_BE  for t = 1, 2,..., 10
*  0 ≤ f_CE(t) ≤ 4 * x_CE  for t = 1, 2,..., 10
*  0 ≤ f_CF(t) ≤ 4 * x_CF  for t = 1, 2,..., 10
*  0 ≤ f_DE(t) ≤ 5 * x_DE  for t = 1, 2,..., 10
*  0 ≤ f_EG(t) ≤ 5 * x_EG  for t = 1, 2,..., 10
*  0 ≤ f_FB(t) ≤ 12 * x_FB for t = 1, 2,..., 10
*  0 ≤ f_FG(t) ≤ 7 * x_FG  for t = 1, 2,..., 10

**3. Flow Conservation at Each Point:**

*  f_AB(t) + f_AC(t) + f_AF(t) = f_BC(t-1) + f_BD(t-1) + f_BE(t-1) + r_B(t-1)  for t = 2, 3,..., 10
*  f_BC(t) + f_CE(t) + f_CF(t) = f_AC(t-1) + r_C(t-1)   for t = 2, 3,..., 10
*  f_BD(t) + f_DE(t) = f_EG(t-1) + r_D(t-1) for t =

In [252]:
print(response4.text)

```python
import pyomo.environ as pyo

# Create a concrete model
model = pyo.ConcreteModel()

# Define time periods
model.T = pyo.RangeSet(1, 10)

# Define pipes
pipes = {
    "AB": 3,
    "AC": 6,
    "AF": 1,
    "BC": 3,
    "BD": 10,
    "BE": 4,
    "CE": 4,
    "CF": 4,
    "DE": 5,
    "EG": 5,
    "FB": 12,
    "FG": 7,
}

# Define points
points = ["B", "C", "D", "E", "F"]

# ==============================================
# Define variables
# ==============================================

# Binary variable for pipe activation status
model.x = pyo.Var(pipes, domain=pyo.Binary)

# Continuous variable for water flow in pipes
model.f = pyo.Var(pipes, model.T, domain=pyo.NonNegativeReals)

# Continuous variable for water released at points
model.r = pyo.Var(points, model.T, domain=pyo.NonNegativeReals)

# ==============================================
# Define constraints
# ==============================================

# 1. Pipe Activation Limit:
def pipe_activation_limit_rule(mo